## LATEST!!

In [ ]:
from PIL import Image, ImageTk
import tkinter as tk
from tkinter import messagebox, ttk
import cv2
import numpy as np
import os
import csv
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

class MakeupRecommendationSystem:
    def __init__(self, root):
        self.root = root
        self.root.title("Makeup Recommendation System")
        self.root.geometry("800x600")

        self.create_ui()

        # Load the background image
        self.original_image = Image.open("homepage.jpeg")

        # Create a label to display the background image
        self.background_label = tk.Label(self.root)
        self.background_label.place(relwidth=1, relheight=1)

        # Bind the <Configure> event to the resize_image function
        self.root.bind("<Configure>", self.resize_image)

        # Create the main heading frame
        self.heading_frame = tk.Frame(self.root, bg="#D2B48C", bd=5)
        self.heading_frame.place(relx=0.5, rely=0.1, relwidth=0.75, relheight=0.1, anchor='n')

        self.heading_label = tk.Label(self.heading_frame, text="Makeup Recommendation System",
                                      font=("Helvetica", 24, "bold"), bg="#D2B48C", fg="white")
        self.heading_label.place(relwidth=1, relheight=1)

        # Create frames for different functionalities
        self.create_function_frame("Start Recognition", 0.4, self.start_recognition)
        self.create_function_frame("User Details", 0.5, self.user_details)
        self.create_function_frame("Visualization", 0.6, self.create_visualization_window)
        self.create_function_frame("Feedback", 0.7, self.give_feedback)

        # Create the exit button frame
        self.exit_frame = tk.Frame(self.root, bg="#D2B48C", bd=5)
        self.exit_frame.place(relx=0.5, rely=0.9, relwidth=0.3, relheight=0.08, anchor='n')

        self.exit_button = tk.Button(self.exit_frame, text="Exit", font=("Helvetica", 18, "bold"), bg="#A0522D",
                                     fg="white", command=self.root.quit)
        self.exit_button.place(relwidth=1, relheight=1)
        self.cap = None

    def resize_image(self, event):
        # Get the current size of the window
        new_width = self.root.winfo_width()
        new_height = self.root.winfo_height()

        # Resize the image to fit the window size
        resized_image = self.original_image.resize((new_width, new_height), Image.LANCZOS)
        self.background_photo = ImageTk.PhotoImage(resized_image)

        # Update the label with the resized image
        self.background_label.config(image=self.background_photo)
        self.background_label.image = self.background_photo

    def create_function_frame(self, text, rel_y, command=None):
        frame = tk.Frame(self.root, bg="#F5DEB3", bd=5)
        frame.place(relx=0.5, rely=rel_y, anchor='n')

        button = tk.Button(frame, text=text, font=("Helvetica", 18, "bold"), bg="#D2B48C", fg="white", command=command)
        button.pack(padx=10, pady=5)

        # Adjust the frame width to fit the button's text
        frame.update_idletasks()
        frame.config(width=button.winfo_width(), height=button.winfo_height())

    def start_recognition(self):
        self.recognition_window = tk.Toplevel(self.root)
        self.recognition_window.title("Skin Tone Detector and Makeup Recommender")
        self.recognition_window.geometry("400x200")
        self.recognition_window.configure(bg="#fae7ce")

        self.lbl_video = tk.Label(self.recognition_window)
        self.lbl_video.pack()

        btn_start = tk.Button(self.recognition_window, text="Start Camera",font=("Helvetica", 14, "bold"),bg="#ffbc85", command=self.start_camera)
        btn_start.pack()

        btn_capture = tk.Button(self.recognition_window, text="Capture Snapshot",font=("Helvetica", 14, "bold"),bg="#ffbc85", command=self.capture_snapshot)
        btn_capture.pack()

        self.recommendations_text = tk.StringVar()
        lbl_recommendations = tk.Label(self.recognition_window, textvariable=self.recommendations_text,font=("Helvetica", 12), justify=tk.LEFT)
        lbl_recommendations.pack()

        btn_quit = tk.Button(self.recognition_window, text="Quit",font=("Helvetica", 14, "bold"),bg="#ffbc85", command=self.recognition_window.destroy)
        btn_quit.pack()

    def start_camera(self):
        self.cap = cv2.VideoCapture(0)
        if not self.cap.isOpened():
            messagebox.showerror("Error", "Could not open webcam.")
            return
        self.update_frame()

    def update_frame(self):
        ret, frame = self.cap.read()
        if ret:
            frame = cv2.flip(frame, 1)
            img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img = Image.fromarray(img)
            img = ImageTk.PhotoImage(img)
            self.lbl_video.imgtk = img
            self.lbl_video.configure(image=img)
            self.lbl_video.after(10, self.update_frame)

    def capture_snapshot(self):
        ret, frame = self.cap.read()
        if ret:
            self.snapshot = cv2.flip(frame, 1)
            self.detect_and_display_recommendations(self.snapshot)

    def detect_and_display_recommendations(self, image):
        undertone = self.get_skin_undertone(image)
        lipstick_recommendations = self.recommend_lipstick(undertone)
        foundation_recommendations = self.recommend_foundation(undertone)
        self.recommendations_text.set(f"Skin Undertone: {undertone}\n"
                                      f"Lipstick: {', '.join(lipstick_recommendations)}\n"
                                      f"Foundation: {', '.join(foundation_recommendations)}")
        self.save_recommendations_to_csv(undertone, lipstick_recommendations, foundation_recommendations)
        messagebox.showinfo("Recommendations Saved", "Recommendations saved to recommendations.csv")

    def get_skin_undertone(self, image):
        hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        lower_skin = np.array([0, 30, 60], dtype=np.uint8)
        upper_skin = np.array([35, 255, 255], dtype=np.uint8)
        mask = cv2.inRange(hsv_image, lower_skin, upper_skin)
        skin = cv2.bitwise_and(image, image, mask=mask)
        lab_image = cv2.cvtColor(skin, cv2.COLOR_BGR2LAB)
        mean_a = np.mean(lab_image[:, :, 1])
        mean_b = np.mean(lab_image[:, :, 2])
        if mean_a > 140 and mean_b > 140:
            return "Warm"
        elif mean_a < 120 and mean_b < 120:
            return "Cool"
        else:
            return "Neutral"

    def recommend_lipstick(self, undertone):
        recommendations = {
            "Warm": ["Peach", "Coral", "Orange"],
            "Cool": ["Berry", "Mauve", "Cherry"],
            "Neutral": ["Rose", "Pink", "Neutral"]
        }

        return recommendations.get(undertone, ["Shade not found"])

    def recommend_foundation(self, undertone):
        recommendations = {
            "Warm": ["Golden Beige", "Honey", "Caramel"],
            "Cool": ["Porcelain", "Ivory", "Sand"],
            "Neutral": ["Buff", "Nude", "Beige"]
        }
        return recommendations.get(undertone, ["Shade not found"])

    def save_recommendations_to_csv(self, undertone, lipstick_recommendations, foundation_recommendations, filename="recommendations.csv"):
        file_exists = os.path.isfile(filename)
        with open(filename, mode='a', newline='') as file:
            writer = csv.writer(file)
            if not file_exists:
                writer.writerow(["Undertone", "Lipstick Recommendations", "Foundation Recommendations"])
            writer.writerow([undertone, ", ".join(lipstick_recommendations), ", ".join(foundation_recommendations)])

    def save_user_details(self, details):
        file_exists = os.path.isfile('userdetails.csv')
        with open('userdetails.csv', mode='a', newline='') as file:
            writer = csv.writer(file)
            if not file_exists:
                writer.writerow(details.keys())
            writer.writerow(details.values())

    def user_details(self):
        def save_details():
            details = {
                'age': age_entry.get(),
                'gender': gender_var.get(),
                'skin_type': skin_type_var.get(),
                'skin_tone': skin_tone_var.get(),
                'allergies': allergies_entry.get(),
                'preferred_style': preferred_style_var.get(),
                'eye_color': eye_color_entry.get()
            }
            self.save_user_details(details)
            messagebox.showinfo("Details Saved", "Users details saved to userdetails.csv")
            details_window.destroy()

        details_window = tk.Toplevel()
        details_window.title("User Details")
        details_window.configure(bg="#fae7ce")
        details_window.geometry("400x600")

        # Increase the font size for all the labels and entries
        font_label = ("Helvetica", 14)
        font_entry = ("Helvetica", 12)

        form_frame = tk.Frame(details_window, padx=20, pady=20, bg="#fae7ce")  # Set background color
        form_frame.pack(padx=10, pady=10, anchor="center")

        # Define font with desired size
        custom_font = ("Helvetica", 12)

        tk.Label(form_frame, text="Age", bg="#fae7ce", font=custom_font).grid(row=0, column=0, sticky="e", padx=5, pady=5)
        age_entry = tk.Entry(form_frame)
        age_entry.grid(row=0, column=1, padx=5, pady=5)

        tk.Label(form_frame, text="Gender", bg="#fae7ce", font=custom_font).grid(row=1, column=0, sticky="e", padx=5, pady=5)
        gender_var = tk.StringVar(details_window)
        tk.OptionMenu(form_frame, gender_var, "Female", "Male").grid(row=1, column=1, padx=5, pady=5)

        tk.Label(form_frame, text="Skin Type", bg="#fae7ce", font=custom_font).grid(row=2, column=0, sticky="e", padx=5, pady=5)
        skin_type_var = tk.StringVar(details_window)
        tk.OptionMenu(form_frame, skin_type_var, "Oily", "Dry", "Combination", "Sensitive").grid(row=2, column=1, padx=5, pady=5)

        tk.Label(form_frame, text="Skin Tone", bg="#fae7ce", font=custom_font).grid(row=3, column=0, sticky="e", padx=5, pady=5)
        skin_tone_var = tk.StringVar(details_window)
        tk.OptionMenu(form_frame, skin_tone_var, "Fair", "Light", "Medium", "Tan", "Deep").grid(row=3, column=1, padx=5, pady=5)

        tk.Label(form_frame, text="Allergies/Sensitivities", bg="#fae7ce", font=custom_font).grid(row=4, column=0, sticky="e", padx=5, pady=5)
        allergies_entry = tk.Entry(form_frame)
        allergies_entry.grid(row=4, column=1, padx=5, pady=5)

        tk.Label(form_frame, text="Preferred Makeup Style", bg="#fae7ce", font=custom_font).grid(row=5, column=0, sticky="e", padx=5, pady=5)
        preferred_style_var = tk.StringVar(details_window)
        tk.OptionMenu(form_frame, preferred_style_var, "Natural", "Glam", "Bold").grid(row=5, column=1, padx=5, pady=5)

        tk.Label(form_frame, text="Eye Color", bg="#fae7ce", font=custom_font).grid(row=6, column=0, sticky="e", padx=5, pady=5)
        eye_color_entry = tk.Entry(form_frame)
        eye_color_entry.grid(row=6, column=1, padx=5, pady=5)

        tk.Button(form_frame, text="Save", command=save_details, font=custom_font).grid(row=7, column=0, columnspan=2, pady=10)

        details_window.mainloop()

    def create_ui(self):
        pass

    def create_visualization_window(self):
        visualization_window = tk.Toplevel(self.root)
        visualization_window.title("Visualization")
        visualization_window.geometry("400x200")
        visualization_window.configure(bg="#fae7ce")

        age_skin_button = tk.Button(visualization_window, text="Age and Skin Type Distribution",font=("Helvetica", 14, "bold"),bg="#ffbc85", command=self.plot_age_skin_type_distribution)
        age_skin_button.pack(pady=10)

        style_button = tk.Button(visualization_window, text="Preferred Makeup Style Distribution",font=("Helvetica", 14, "bold"),bg="#ffbc85", command=self.plot_preferred_style_pie_chart)
        style_button.pack(pady=10)



    def create_ui(self):
        age_skin_button = tk.Button(self.root, text="Age and Skin Type Distribution", command=self.plot_age_skin_type_distribution)
        age_skin_button.pack(pady=10)

        style_button = tk.Button(self.root, text="Preferred Makeup Style Distribution", command=self.plot_preferred_style_pie_chart)
        style_button.pack(pady=10)

    def plot_age_skin_type_distribution(self):
        try:
            data = pd.read_csv('userdetails.csv', names=['age', 'gender', 'skin_type', 'skin_tone', 'allergies', 'preferred_style', 'eye_color'])
            if data is not None and not data.empty:
                # Create a new window for the visualization
                age_skin_window = tk.Toplevel(self.root)
                age_skin_window.title("Age and Skin Type Distribution")
                age_skin_window.geometry("800x600")

                plt.figure(figsize=(12, 8))
                sns.countplot(x='age', hue='skin_type', data=data, palette='Set2')
                plt.title('Age and Skin Type Distribution')
                plt.xlabel('Age')
                plt.ylabel('Count')
                plt.legend(title='Skin Type')

                # Create a canvas for the plot
                canvas = FigureCanvasTkAgg(plt.gcf(), master=age_skin_window)
                canvas.draw()
                canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)
            else:
                print("No data to display.")
        except FileNotFoundError:
            print("The file 'userdetails.csv' was not found.")
        except pd.errors.EmptyDataError:
            print("The file 'userdetails.csv' is empty.")
        except Exception as e:
            print(f"An error occurred: {e}")

    def plot_preferred_style_pie_chart(self):
        visualization_window = tk.Toplevel(self.root)
        visualization_window.title("Preferred Makeup Style Distribution")
        visualization_window.geometry("800x600")

        try:
            data = pd.read_csv('userdetails.csv', names=['age', 'gender', 'skin_type', 'skin_tone', 'allergies', 'preferred_style', 'eye_color'])

            # Count the occurrences of each preferred style
            style_counts = data['preferred_style'].value_counts()

            fig, ax = plt.subplots(figsize=(8, 8))
            ax.pie(style_counts, labels=style_counts.index, autopct='%1.1f%%', startangle=90, colors=plt.cm.Paired(range(len(style_counts))))
            ax.set_title('Preferred Makeup Style Distribution')

            # Create a canvas for the plot
            canvas = FigureCanvasTkAgg(fig, master=visualization_window)
            canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)

            canvas.draw()
        except FileNotFoundError:
            messagebox.showerror("Error", "The file 'userdetails.csv' was not found.")
        except pd.errors.EmptyDataError:
            messagebox.showerror("Error", "The file 'userdetails.csv' is empty.")
        except Exception as e:
            messagebox.showerror("Error", f"An error occurred: {e}")

    def visualization(self):
        self.plot_age_skin_type_distribution()

    def give_feedback(self):
        feedback_window = tk.Toplevel(self.root)
        feedback_window.title("Feedback")
        feedback_window.configure(bg="#fae7ce")

        feedback_label = tk.Label(feedback_window, text="We'd love to hear your feedback!", font=("Helvetica", 16),bg="#fae7ce")
        feedback_label.pack(pady=10)

        feedback_text = tk.Text(feedback_window, height=10, width=50)
        feedback_text.pack(pady=10)

        submit_button = tk.Button(feedback_window, text="Submit",font=("Helvetica", 14, "bold"),bg="#ffbc85", command=lambda: self.save_feedback(feedback_text.get("1.0", "end-1c")))
        submit_button.pack(pady=10)

    def save_feedback(self, feedback, filename="feedback.csv"):
        with open(filename, mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([feedback])
        messagebox.showinfo("Feedback Saved", "Thank you for your feedback!")

    def exit_application(self):
        self.root.destroy()



if __name__ == "__main__":
    root = tk.Tk()
    app = MakeupRecommendationSystem(root)
    root.mainloop()
